# NLP

## Explore Tiny Shakespear dataset

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("Length of the dataset in characters: ", len(text))

Length of the dataset in characters:  1115393


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Number of unique characters: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique characters: 65


## Tokenizer

In [10]:
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [14]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## Dataset Split

In [17]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
block_size = 8 # or context length
train_data[:block_size +1 ]
# It is nine characters because then the model can predict the next character 8 times

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context} the target: {target}")

When the input is tensor([18]) the target: 47
When the input is tensor([18, 47]) the target: 56
When the input is tensor([18, 47, 56]) the target: 57
When the input is tensor([18, 47, 56, 57]) the target: 58
When the input is tensor([18, 47, 56, 57, 58]) the target: 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [27]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
----
when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 

In [56]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # because no targets -> Loss is None -> logits still have to be permuted -> becomes (B, C)
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1) # Es gibt ja jetzt für jeden Token eine softmax wahrscheinlichkeit
            # torch.multinomial nimmt jetzt zufällig einen dieser Token with respect to their wahrscheinlichkeit
            # es ist also am wahrscheinlichsten, dass der Token mit der höchsten probability genommen wird, aber es können auch
            # unwahrscheinlichere gewählt werden, was die Ausgabe natürlicher macht
            # zum beispiel: "Die Katze liegt auf dem " -> Sofa, Bett, Boden -> Greedy würde immer das selbe wählen
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
        

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
#print(logits.shape) # get the predictions for every 
#print(loss)

In [57]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),max_new_tokens = 100)[0].tolist())) # 0 is the new line character


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [59]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [85]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4394798278808594


In [90]:
print(decode(m.generate(torch.zeros((1,1), dtype = torch.long),max_new_tokens = 300)[0].tolist()))


I me:


my id, s
S:

Mayo, m.
BUTHERO:
See cect borenghis.
Mory o hisip woow walim,
CULBUERDe ce tow hefrwary theds
thto y, w t an and:
Fr ik's,
FF ldot gofrtarrurndse mbashech.
Aprixeeave, ok.
S al tenowory m:
Fithen t cot sut d thaiday, uisitry whesougus sllle igu h w vevoshet olnje ndous; medewil


## The mathematical trick in self-attention

In [94]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [95]:
xbow = torch.zeros((B,T,C)) # bow = bag of words (term that people use when you average the words)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)

# BUT its very inefficient -> Marix multiplication helps

In [96]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [97]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [104]:
# Version 2
wei = torch.tril(torch.ones(T, T)) # wei = weights
wei = wei/wei.sum(1, keepdim = True)
xbow2 = wei @ x # (T,T) @ (B, T, C) -> Pytorch will firs extend the first matrix to a batch dimension
# (B, T, T) @ (B, T, C) -> For each batch element --> Result: (B, T, C)
torch.allclose(xbow, xbow2)

True

In [106]:
# Version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1) # -> produces the averaging matrix
xbow3= wei @ x
torch.allclose(xbow, xbow3)

True

In [100]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [101]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [114]:
# Version 4: Self-Attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# Single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)

k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2, -1) # do not transpose batch dimension 
# (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [113]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [115]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [116]:
k.var()

tensor(1.0449)

In [117]:
q.var()

tensor(1.0700)

In [119]:
wei.var() #Scaling keeps variance at one, without scaling softmax converges to one hot encoding when high weights are generated 

tensor(1.0918)

In [120]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [121]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [122]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-3.5763e-09), tensor(1.0000))

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 8
block_size = 32  # Context Length
max_iters = 1000
eval_interval = 500
learning_rate = 1e-3 #3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64 # needs to be divisible by n_head
n_head = 8
n_layer = 6
dropout = 0.2
save_path = 'model_exp.pth'
# --------------------

torch.manual_seed(1337)

with open('news-commentary-v10.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad() # because we are not using backward, makes pytorch faster
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model (X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ One Head of Self-Attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets = None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:] # because no targets -> Loss is None -> logits still have to be permuted -> becomes (B, C)
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1) # Es gibt ja jetzt für jeden Token eine softmax wahrscheinlichkeit
            # torch.multinomial nimmt jetzt zufällig einen dieser Token with respect to their wahrscheinlichkeit
            # es ist also am wahrscheinlichsten, dass der Token mit der höchsten probability genommen wird, aber es können auch
            # unwahrscheinlichere gewählt werden, was die Ausgabe natürlicher macht
            # zum beispiel: "Die Katze liegt auf dem " -> Sofa, Bett, Boden -> Greedy würde immer das selbe wählen
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [27]:

#model = BigramLanguageModel()
#model.load_state_dict(torch.load('model.pth'))

model = BigramLanguageModel()
model.load_state_dict(torch.load('model_3.pth'))

/var/folders/97/07fn6f7n48j71zk65dskp3gm0000gn/T/ipykernel_2627/4146350656.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_3.pth'

RuntimeError: Error(s) in loading state_dict for BigramLanguageModel:
	Unexpected key(s) in state_dict: "blocks.0.sa.heads.4.tril", "blocks.0.sa.heads.4.key.weight", "blocks.0.sa.heads.4.query.weight", "blocks.0.sa.heads.4.value.weight", "blocks.0.sa.heads.5.tril", "blocks.0.sa.heads.5.key.weight", "blocks.0.sa.heads.5.query.weight", "blocks.0.sa.heads.5.value.weight", "blocks.0.sa.heads.6.tril", "blocks.0.sa.heads.6.key.weight", "blocks.0.sa.heads.6.query.weight", "blocks.0.sa.heads.6.value.weight", "blocks.0.sa.heads.7.tril", "blocks.0.sa.heads.7.key.weight", "blocks.0.sa.heads.7.query.weight", "blocks.0.sa.heads.7.value.weight", "blocks.1.sa.heads.4.tril", "blocks.1.sa.heads.4.key.weight", "blocks.1.sa.heads.4.query.weight", "blocks.1.sa.heads.4.value.weight", "blocks.1.sa.heads.5.tril", "blocks.1.sa.heads.5.key.weight", "blocks.1.sa.heads.5.query.weight", "blocks.1.sa.heads.5.value.weight", "blocks.1.sa.heads.6.tril", "blocks.1.sa.heads.6.key.weight", "blocks.1.sa.heads.6.query.wei

In [23]:
context = torch.tensor(encode('Eierkuchen '), dtype = torch.long)
context = context.unsqueeze(dim = 0)
print(context.tolist())

[[32, 65, 61, 74, 67, 77, 59, 64, 61, 70, 1]]


In [24]:
print(decode(context.squeeze().tolist()))

Eierkuchen 


In [25]:
print(decode(model.generate(context, max_new_tokens = 500)[0].tolist()))

NameError: name 'model' is not defined

In [3]:
import torch


In [2]:
print(decode(model.generate(context, max_new_tokens = 500)[0].tolist()))

NameError: name 'decode' is not defined

In [11]:
from bigram import BigramLanguageModel


In [12]:
checkpoint = torch.load('model_1.pth', map_location=torch.device('cpu'))
config = checkpoint['config']
print(config)
model = BigramLanguageModel(**config)
model.load_state_dict(checkpoint['model_state_dict'])

{'vocab_size': 311, 'block_size': 256, 'n_embd': 384, 'n_head': 6, 'n_layer': 6}


/var/folders/97/07fn6f7n48j71zk65dskp3gm0000gn/T/ipykernel_52332/3792025281.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model_1.pth', map_lo

<All keys matched successfully>

In [13]:
def load_model_with_config(save_path):
    checkpoint = torch.load(save_path, map_location=torch.device('cpu'))
    config = checkpoint['config']
    model = BigramLanguageModel(**config)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

In [14]:
model = load_model_with_config('model_1.pth')

/var/folders/97/07fn6f7n48j71zk65dskp3gm0000gn/T/ipykernel_52332/1855571868.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path, map_locati

In [15]:
with open('datasets/news-commentary-v10.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [16]:
context = torch.tensor(encode('Wir laufen in ein Haus '), dtype = torch.long)
context = context.unsqueeze(dim = 0)
print(context.tolist())

[[54, 71, 80, 1, 74, 63, 83, 68, 67, 76, 1, 71, 76, 1, 67, 71, 76, 1, 39, 63, 83, 81, 1]]


In [19]:
print(decode(model.generate(context, max_new_tokens = 500)[0].tolist()))

Wir laufen in ein Haus dienen die iranische Uneinigkeit verstoßen, einen welttretigen sträubenden Reis allerdings in Frage, ob ein Patrouillen an die Akten in der Leitung gab, die wiederholt mehr besonders gemeinsam dramatisch gebraucht sind.
Die Vereinigten Staaten und Zentralregierungen in Glattsindikatoren, insbesondere in Harvard und besondern dafür ist Syriens ideale Systeme, die die immer mehr Chen Olieferung und Gesellschaften zur Überwindung von Arabischen Union im Sinne hinterhergestellt hatten.
Als Bürgerrec


In [125]:
from lyricsgenius import Genius

In [126]:
API = '0X82crAR7oOXruvuM83D4A-dtzSoAY2g5l4dTeUooLmhQhpD2DISXXJDkuTYLx6m'

lyrics = []
genius = Genius(API)
genius.remove_section_headers = False

In [161]:
artist = genius.search_artist("$uicideboy$")

Searching for songs by $uicideboy$...

Song 1: "Kill Yourself (Part III)"
Song 2: "Paris"
Song 3: "Antarctica"
Song 4: "$outh $ide $uicide"
Song 5: "2nd Hand"
Song 6: "I No Longer Fear the Razor Guarding My Heel"
Song 7: "Carrollton"
Song 8: "New Chains, Same Shackles"
Song 9: "...And to Those I Love, Thanks for Sticking Around"
Song 10: "O Pana!"
Song 11: "Memoirs of a Gorilla"
Song 12: "For the Last Time"
Song 13: "Kill Yourself (Part IV)"
Song 14: "Magazine"
Song 15: "LTE"
Song 16: "Fuckthepopulation"
Song 17: "Low Key"
Song 18: "Champion of Death"
Song 19: "Mount Sinai"
Song 20: "All Known $UICIDEBOY$ Alter-Egos and Personas"
Song 21: "DIEMONDS"
Song 22: "Now I’m Up to My Neck With Offers"
Song 23: "122 Days"
Song 24: "Exodus"
Song 25: "Do You Believe in God?"
Song 26: "I No Longer Fear the Razor Guarding My Heel (IV)"
Song 27: "You’re Now Tuning in to 66.6 FM with DJ Rapture (The Hottest Hour of the Evening)"
Song 28: "I No Longer Fear the Razor Guarding My Heel (III)"
Song 29: "A

In [162]:
artist.save_lyrics()

Lyrics_uicideboy.json already exists. Overwrite?
(y/n):  y


Wrote Lyrics_uicideboy.json.


In [1]:
import json

with open('Lyrics_uicideboy.json', 'r') as file:
    data = json.load(file)

In [219]:
with open('datasets/concatenated_lyrics.txt', 'w', encoding='utf-8') as file:
    for i in range(len(data['songs'])):
        title = data['songs'][i]['title']
        lyrics = data['songs'][i]['lyrics']
        start = lyrics.find(title)
        end = lyrics.find("Embed")
        new_lyrics = lyrics[start:end-3]
        remove_phrases = ["See $uicideboy$ Live", "Get tickets", "You might also like", "might also li", "as low as"]

        for phrase in remove_phrases:
            new_lyrics = new_lyrics.replace(phrase, "")

        #sentence_remove = "See $uicideboy$ LiveGet tickets as low as $82You might also like"
        #new_lyrics = new_lyrics.replace(sentence_remove,"")
        file.write(new_lyrics + '\n\n')

In [5]:
with open('concatenated_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

691866


In [6]:
print(chars)

['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '\xa0', 'á', 'â', 'é', 'ì', 'í', 'ó', 'œ', 'Κ', 'Π', 'ά', 'ί', 'α', 'γ', 'ε', 'η', 'κ', 'λ', 'μ', 'ν', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'ό', 'В', 'М', 'С', 'а', 'в', 'г', 'д', 'е', 'и', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ч', 'ь', 'я', 'ἐ', 'ἡ', 'ὁ', 'ὐ', 'ῖ', 'ῶ', '\u2005', '\u200b', '–', '—', '‘', '’', '“', '”', '•', '\u205f', '\u2060', '↗', '✧', '中', '文', '翻', '译']


In [155]:
print('\u205f')

In [32]:
a.find("Hello")

4

In [4]:
import torch

In [17]:
def load_model_with_config(save_path):
    checkpoint = torch.load(save_path, map_location=torch.device('cpu'))
    config = checkpoint['config']
    model = BigramLanguageModel(**config)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

In [19]:
model1 = load_model_with_config('SB_model_1.pth')
model2 = load_model_with_config('SB_model_2.pth')

/var/folders/97/07fn6f7n48j71zk65dskp3gm0000gn/T/ipykernel_52332/1855571868.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path, map_locati

In [20]:
with open('datasets/concatenated_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) } # string to integer
itos = { i:ch for i,ch in enumerate(chars) } # integer to string
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [23]:
context = torch.tensor(encode('We smoke a J full of Weed \n '), dtype = torch.long)
context = context.unsqueeze(dim = 0)
print(context.tolist())

[[50, 61, 1, 75, 69, 71, 67, 61, 1, 57, 1, 37, 1, 62, 77, 68, 68, 1, 71, 62, 1, 50, 61, 61, 60, 1, 0, 1]]


In [24]:
print(decode(model1.generate(context, max_new_tokens = 512)[0].tolist()))

We smoke a J full of Weed 
 he smoke a bitch that play hard (Chush it up)
Told that bitch, "Get out my ear," she need to chill and— (Hit the blunt)
Xanax bars up in my drink, before I pour 'em— (Crush it up)
We never fuck with you, fuckboy, don't act like you can— (Hit the blunt)
Take that motherfuckin' bat upside your skull and— (Crush it Up)
So full of them drugs, I might throw up, I knew I shouldn'ta— (Hit the blunt)
[Outro: JGRXXN & Gimisum Family]
OxyContin, some Lortabs, crush it up
Hit the blunt, hit the blunt, crush it up
OxyC


In [25]:
print(decode(model2.generate(context, max_new_tokens = 512)[0].tolist()))

We smoke a J full of Weed 
  $82[Chorus]
Mike a bitch, pick oh, we pick oh
Mike a bitch, cered of that motherfucker that motherfuckin' with
Creepin' out the smoke a cross traight of spot

[Verse 2: YUNG CHRIST]
Creepin' out the snakes, buckle dick, count with mud
Crucified with the on my mouth
Crucified waways, get buckle
Keepin' out the crime, keepin' the pistol green live up and therfuckin' cose
Crucified with the Files with the Tammy!"
Hear "Let the Uzi," fuck her before whore?" I need to surrounded at the $uicide to dour
Never liq
